# Consume and Ingest Data via SAP Datasphere Open SQL Schemas

## Purpose
This notebook helps you to get started working with SAP Datasphere Open SQL schemas using the [hdbcli](https://pypi.org/project/hdbcli/) and  [SQLAlchemy](https://www.sqlalchemy.org/) packages. It will:
1. [Connect to your Open SQL Schema](#connect) using the credentials provided by you in the [db_user.json](./db_user.json) file.
2. [Read data from a view exposed for consumption by the space your schema is attached to ](#read-data)
3. [Ingest data from any csv files in your script folder](#ingest-data)

This script has been tested and reviewed by SAP but, in case of errors or other problems, SAP is not liable to offer fixes nor any kind of support and maintenance. It is recommended that you test the script first, ideally in a test environment. You can also edit, enhance, copy or otherwise use the script in your own projects.

## Prerequisites
You must:
- [Have an SAP Datasphere database user for a space exposing data](https://help.sap.com/docs/SAP_DATASPHERE/be5967d099974c69b77f4549425ca4c0/798e3fd6707940c3bd2219b2d1ebaac2.html).
- [Obtain the credentials for an OAuth client for the specified tenant](https://help.sap.com/docs/SAP_DATASPHERE/9f804b8efa8043539289f42f372c4862/3f92b46fe0314e8ba60720e409c219fc.html).


## Storing Credentials
For simplicity, the SAP Datasphere database user credentials used in this script are stored in a plain-text file. When adapting this script you should use your organization's credentials store solution and apply any other security recommendations.

## More Information
For detailed information about working with Open SQL Schemas, see: [Integrating Data via Database Users/Open SQL Schemas](https://help.sap.com/docs/SAP_DATASPHERE/be5967d099974c69b77f4549425ca4c0/3de55a78a4614deda589633baea28645.html) in the SAP Datasphere documentation.

<a id='connect'></a>
## Connect to Your Open SQL Schema
Using the credentials specified in the [db_user.json](./db_user.json) file.

In [ ]:
# Import packages
import json
from hdbcli import dbapi # To connect to SAP HANA Database Open SQL schema
from sqlalchemy import create_engine
from sqlalchemy.engine import reflection
import sqlalchemy.types
import glob
import pandas as pd

# Get db user details from "db_user.json" file
with open('db_user.json', 'r') as file:
    data = file.read()
    db_user = json.loads(data)
    print(f"Connecting with db user {db_user['name']}.")
    
#Initialize your connection
conn = dbapi.connect(
    address=db_user['host'],
    port=db_user['port'],
    user=db_user['name'],
    password=db_user['password'], 
    encrypt=True, # must be set to True when connecting to HANA Cloud
    sslValidateCertificate=False # True HC, False for HANA Express.
)
#If no errors, print connected
print('Connected.')

<a id='read-data'></a>
## Read Data Exposed by the Space
Read data from the exposed view in the space schema as specified in the [db_user.json](./db_user.json) file.

In [ ]:
# Using official SAP HANA client, hdbcli
dbSpaceSchema = db_user['dbSpaceSchema']
dbExposedView = db_user['dbExposedView']
cursor = conn.cursor()
sql_command = f'select top 2 * from "{dbSpaceSchema}"."{dbExposedView}"'
print(sql_command)
cursor.execute(sql_command)
rows = cursor.fetchall()
for row in rows:
    for col in row:
        print ("%s" % col, end=" ")
    print (" ")

<a id='ingest-data'></a>
## Ingest Data to Your Schema
Load data from any csv files in your script folder to create tables in your Open SQL schema.

In [ ]:
# Create connection string using sqlalchemy
connect_string = f"hana://{db_user['name']}:{db_user['password']}@{db_user['host']}:{db_user['port']}"

# Connect to Open SQL schema
engine = create_engine(connect_string, connect_args={
    'sslTrustStore' : "",
    'encrypt' : 'true',
    'sslHostNameInCertificate' : '*'
}, echo=True)                                    # To enable verbose debugging information
connection = engine.connect()
# inspector = reflection.Inspector.from_engine(engine)
inspector = sqlalchemy.inspect(engine)

# Get a list of all CSV files in the folder
csv_files = glob.glob('*.csv')
csv_files_no = 0

# Load each CSV file to a dataframe, and then load the data to the Open SQL Database
for file in csv_files:
    table_name = file.removesuffix('.csv')
    print(f'Loading {table_name}')
    df = pd.read_csv(file,sep=';')
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    csv_files_no += 1
print(f'Loaded {csv_files_no} csv files.')